# Biochar credits crosswalk notebook (7 standards)

Modeling sandbox to compare how many credits would be minted for the *same theoretical biochar project* under:
- Global Biochar C-Sink (Carbon Standards International)
- Global Artisan C-Sink (Carbon Standards International)
- Riverse / Rainbow Standard (BiCRS – Biochar to soils)
- Puro.earth (Biochar Methodology)
- Isometric (Biochar Protocol)
- Climate Action Reserve (U.S. & Canada Biochar Protocol)
- Verra VCS (VM0044)

Notes:
- Standards evolve; this notebook is transparent and modular.
- Where a standard relies on external tools (e.g., Ecoinvent or proprietary calculators), you can paste those emissions outputs here.


## 0) Setup

In [ ]:
\
import math
from dataclasses import dataclass, asdict
from typing import Optional, Dict, Any
import pandas as pd

def tCO2_from_tC(tC: float) -> float:
    return tC * (44/12)

def dry_mass_from_wet(wet_t: float, moisture_frac: float) -> float:
    # moisture_frac = mass water / total wet mass (0-1)
    return wet_t * (1 - moisture_frac)


## 1) Define project inputs

In [ ]:
\
@dataclass
class ProjectInputs:
    # Biochar quantities
    biochar_wet_t: float                 # tonnes wet biochar delivered/applied in the period
    biochar_moisture_frac: float         # 0-1
    biochar_Corg_frac: float             # 0-1 (organic carbon fraction on dry basis)
    biochar_HCorg_molar: float           # molar H/Corg

    # Context for soil applications
    soil_temp_C: float                   # mean annual soil temp at application site

    # Emissions (enter if known; else keep 0 to compare storage-side)
    E_biomass_tCO2e: float = 0.0
    E_production_tCO2e: float = 0.0
    E_transport_tCO2e: float = 0.0
    E_use_tCO2e: float = 0.0
    E_leakage_tCO2e: float = 0.0
    C_baseline_tCO2e: float = 0.0

    # Optional: random reflectance pathway inputs
    inertinite_fraction: Optional[float] = None  # fraction (0-1) of residual C with R0 >= 2%
    residual_C_fraction: Optional[float] = None  # residual organic carbon fraction (0-1)

inputs = ProjectInputs(
    biochar_wet_t=1000.0,
    biochar_moisture_frac=0.15,
    biochar_Corg_frac=0.75,
    biochar_HCorg_molar=0.30,
    soil_temp_C=15.0,
    # emissions placeholders (edit)
    E_biomass_tCO2e=0.0,
    E_production_tCO2e=0.0,
    E_transport_tCO2e=0.0,
    E_use_tCO2e=0.0,
    E_leakage_tCO2e=0.0,
    C_baseline_tCO2e=0.0,
    # optional for R0 pathway
    inertinite_fraction=0.72,
    residual_C_fraction=0.95,
)

inputs


## 2) Shared physical quantities

In [ ]:
\
def shared_physical(inputs: ProjectInputs) -> Dict[str, float]:
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    tC0 = dry_t * inputs.biochar_Corg_frac
    tCO2e0 = tCO2_from_tC(tC0)
    return {
        "biochar_dry_t": dry_t,
        "Corg_tC": tC0,
        "Corg_tCO2e": tCO2e0
    }

shared_physical(inputs)


## 3) Standard calculators

In [ ]:
\
# --------------------------
# 3.1 CSI: Global Artisan C-Sink
# --------------------------
def calc_csi_artisan(inputs: ProjectInputs,
                     safety_margin: float = 0.03,
                     persistence_100y: float = 0.74) -> Dict[str, Any]:
    '''
    Worked example in CSI Global Artisan C-Sink:
      credits = dry_mass * Corg * (1 - 3% margin) * 44/12 * 0.74 persistence (soil).
    '''
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    csink_potential = dry_t * inputs.biochar_Corg_frac * (1 - safety_margin) * (44/12)
    credits = csink_potential * persistence_100y
    return {
        "credits_tCO2e": credits,
        "csink_potential_tCO2e": csink_potential,
        "assumptions": {"safety_margin": safety_margin, "persistence_100y": persistence_100y},
        "notes": "Artisan: dry_mass*Corg*(1-3%)*44/12*0.74"
    }

# --------------------------
# 3.2 CSI: Global Biochar C-Sink (soil; simplified issuance proxy)
# --------------------------
def calc_csi_global_csink(inputs: ProjectInputs) -> Dict[str, Any]:
    '''
    CSI defines C-Sink_0 = Corg * dry_mass (tC), and C-Sink_H metrics (average storage over horizon H).
    As a transparent proxy for 'credits minted', scale initial storage using Table 1 factors for soil:
      - H/Corg <= 0.4: C-Sink_100 approx 88.1 for 100 tCO2e => factor 0.881
      - H/Corg > 0.4 : C-Sink_100 approx 52.5 for 100 tCO2e => factor 0.525
    Use CSI calculators for exact annualized matching.
    '''
    initial = shared_physical(inputs)["Corg_tCO2e"]
    if inputs.biochar_HCorg_molar <= 0.4:
        factor_100y, regime = 0.881, "H/Corg <= 0.4 (PAC75%+SPC25%)"
    else:
        factor_100y, regime = 0.525, "H/Corg > 0.4 (SPC100%)"
    return {
        "credits_tCO2e": initial * factor_100y,
        "initial_storage_tCO2e": initial,
        "factor_100y": factor_100y,
        "regime": regime,
        "notes": "Proxy: initial_tCO2e * factor_100y"
    }

# --------------------------
# 3.3 Riverse / Rainbow (BiCRS - Biochar to soils) Approach 1 (100y, H/Corg)
# --------------------------
def _rainbow_coeffs(soil_temp_C: float):
    if soil_temp_C < 7.49:
        return 1.13, 0.46
    if soil_temp_C < 12.49:
        return 1.10, 0.59
    if soil_temp_C < 17.49:
        return 1.04, 0.64
    if soil_temp_C < 22.49:
        return 1.01, 0.65
    return 0.98, 0.66

def calc_riverse_rainbow_100y(inputs: ProjectInputs) -> Dict[str, Any]:
    '''
    Rainbow biochar-to-soils module:
      Fperm100 = c - m*H/Corg
      RP_storage100 = Fperm100*Corg*Abiochar*(1-M%)*44/12
    Here we subtract user-provided emissions totals as a convenience.
    '''
    c, m = _rainbow_coeffs(inputs.soil_temp_C)
    Fperm = max(0.0, min(1.0, c - m * inputs.biochar_HCorg_molar))
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    stored = Fperm * inputs.biochar_Corg_frac * dry_t * (44/12)
    net = stored - (inputs.E_biomass_tCO2e + inputs.E_production_tCO2e + inputs.E_transport_tCO2e + inputs.E_use_tCO2e + inputs.E_leakage_tCO2e)
    return {"credits_tCO2e": net, "storage_component_tCO2e": stored, "Fperm100": Fperm, "coeffs": {"c": c, "m": m}}

# --------------------------
# 3.4 Puro.earth Biochar (Edition 2025 v2)
# --------------------------
def calc_puro_2025(inputs: ProjectInputs) -> Dict[str, Any]:
    '''
    Puro 2025:
      CORCs = Cstored - Cbaseline - Closs - Eproject - Eleakage
      Cstored = Qbiochar*Corg*44/12
      Closs = Cstored*(100-PF)/100
      PF = M - a*H/Corg, where M,a depend on soil temperature (rounded up, min 7C)
    This notebook includes M,a lookups for Ts=7..17C (extend as needed).
    '''
    Ts = max(7, math.ceil(inputs.soil_temp_C))
    M_lookup = {7: 13.73, 8: 13.58, 9: 13.43, 10: 13.29, 11: 13.15, 12: 13.01, 13: 12.88, 14: 12.75, 15: 12.62, 16: 12.49, 17: 12.37}
    a_lookup = {7: 42.52, 8: 43.10, 9: 43.67, 10: 44.21, 11: 44.74, 12: 45.26, 13: 45.77, 14: 46.27, 15: 46.77, 16: 47.27, 17: 47.76}
    Ts_cap = min(Ts, 17)
    M, a = M_lookup[Ts_cap], a_lookup[Ts_cap]
    PF_percent = max(0.0, min(100.0, M - a * inputs.biochar_HCorg_molar))
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    Cstored = dry_t * inputs.biochar_Corg_frac * (44/12)
    Closs = Cstored * (100 - PF_percent) / 100.0
    Eproject = inputs.E_biomass_tCO2e + inputs.E_production_tCO2e + inputs.E_transport_tCO2e + inputs.E_use_tCO2e
    Eleakage = inputs.E_leakage_tCO2e
    CORCs = Cstored - inputs.C_baseline_tCO2e - Closs - Eproject - Eleakage
    return {"credits_tCO2e": CORCs, "Cstored_tCO2e": Cstored, "Closs_tCO2e": Closs, "PF_percent": PF_percent, "Ts_used_C": Ts_cap}

# --------------------------
# 3.5 Isometric (proxy)
# --------------------------
def calc_isometric(inputs: ProjectInputs, durability_years: int = 200) -> Dict[str, Any]:
    '''
    Isometric describes:
      - 200y durability: H/Corganic + projected decay curves
      - 1000y durability: Random Reflectance; inertinite fraction credited
    We implement transparent proxies:
      - 200y: Woolf-style Fperm (placeholder)
      - 1000y: dry_mass*Corg*residual_C*inertinite_fraction*44/12
    '''
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    if durability_years == 1000:
        if inputs.inertinite_fraction is None or inputs.residual_C_fraction is None:
            raise ValueError("Need inertinite_fraction and residual_C_fraction for 1000-year pathway.")
        stored = dry_t * inputs.biochar_Corg_frac * inputs.residual_C_fraction * inputs.inertinite_fraction * (44/12)
    else:
        c, m = _rainbow_coeffs(inputs.soil_temp_C)
        Fperm = max(0.0, min(1.0, c - m * inputs.biochar_HCorg_molar))
        stored = Fperm * inputs.biochar_Corg_frac * dry_t * (44/12)
    net = stored - (inputs.E_biomass_tCO2e + inputs.E_production_tCO2e + inputs.E_transport_tCO2e + inputs.E_use_tCO2e + inputs.E_leakage_tCO2e)
    return {"credits_tCO2e": net, "storage_component_tCO2e": stored, "durability_years": durability_years}

# --------------------------
# 3.6 CAR (simplified single biochar type/end use)
# --------------------------
def calc_car(inputs: ProjectInputs, PEU: Optional[float] = None) -> Dict[str, Any]:
    '''
    CAR Eq 5.12 (simplified single biochar type/end use):
      PC = Mb * DM * OCb * PEU * 3.67
    If PEU is not provided, we approximate PEU with Woolf-style Fperm (placeholder).
    '''
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)  # Mb*DM
    OCb = inputs.biochar_Corg_frac
    if PEU is None:
        c, m = _rainbow_coeffs(inputs.soil_temp_C)
        PEU = max(0.0, min(1.0, c - m * inputs.biochar_HCorg_molar))
    PC = dry_t * OCb * PEU * (44/12)
    net = PC - (inputs.E_biomass_tCO2e + inputs.E_production_tCO2e + inputs.E_transport_tCO2e + inputs.E_use_tCO2e + inputs.E_leakage_tCO2e)
    return {"credits_tCO2e": net, "storage_component_tCO2e": PC, "PEU": PEU}

# --------------------------
# 3.7 Verra VM0044 (simplified)
# --------------------------
def calc_verra_vm0044(inputs: ProjectInputs, PRde: Optional[float] = None) -> Dict[str, Any]:
    '''
    VM0044 v1.2 (simplified):
      CC = M * FCp * PRde
      ERPS = (CC*44/12) - PE_production
    Full net includes baseline, leakage, and reversal risk mitigation.
    '''
    dry_t = dry_mass_from_wet(inputs.biochar_wet_t, inputs.biochar_moisture_frac)
    FCp = inputs.biochar_Corg_frac
    if PRde is None:
        PRde = 0.80  # placeholder (medium-temp default in VM0044 Table 3)
    CC = dry_t * FCp * PRde  # tonnes C
    ERPS = tCO2_from_tC(CC) - inputs.E_production_tCO2e
    net = ERPS - (inputs.E_biomass_tCO2e + inputs.E_transport_tCO2e + inputs.E_use_tCO2e + inputs.E_leakage_tCO2e)
    return {"credits_tCO2e": net, "ERPS_tCO2e": ERPS, "PRde": PRde}


## 4) Run and compare

In [ ]:
\
results = [
    ("CSI Global Biochar C-Sink (proxy)", calc_csi_global_csink(inputs)),
    ("CSI Global Artisan C-Sink", calc_csi_artisan(inputs)),
    ("Riverse / Rainbow (100y, H/C)", calc_riverse_rainbow_100y(inputs)),
    ("Puro.earth (2025 v2)", calc_puro_2025(inputs)),
    ("Isometric (200y proxy)", calc_isometric(inputs, durability_years=200)),
    ("Isometric (1000y proxy)", calc_isometric(inputs, durability_years=1000)),
    ("CAR (simplified)", calc_car(inputs)),
    ("Verra VM0044 (simplified)", calc_verra_vm0044(inputs)),
]

df = pd.DataFrame([{"standard": n, **o} for n,o in results])
df[["standard","credits_tCO2e"]].sort_values("credits_tCO2e", ascending=False)


## 5) Sensitivity sweep (H/Corg and soil temperature)

In [ ]:
\
import numpy as np

def sweep(hc_values=np.linspace(0.1,0.7,13), temps=[8,12,15,20,25]):
    rows = []
    base = asdict(inputs)
    for Ts in temps:
        for hc in hc_values:
            base["soil_temp_C"] = float(Ts)
            base["biochar_HCorg_molar"] = float(hc)
            inp = ProjectInputs(**base)
            rows.append({
                "soil_temp_C": Ts,
                "H_Corg": hc,
                "Puro_CORCs": calc_puro_2025(inp)["credits_tCO2e"],
                "Rainbow_100y": calc_riverse_rainbow_100y(inp)["credits_tCO2e"],
                "CAR_storage": calc_car(inp)["storage_component_tCO2e"],
            })
    return pd.DataFrame(rows)

df_sweep = sweep()
df_sweep.head()


## Interactive explorer (for Voila)
Use the controls to explore how credits vary with H/Corg and soil temperature.

This section is designed to render nicely in **Voila** (code hidden).

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, Checkbox, HBox, VBox, Output, Layout
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# Ensure df_sweep exists
try:
    df_sweep
except NameError:
    df_sweep = sweep()

out = Output()

metrics = {
    "Puro (CORCs)": "Puro_CORCs",
    "Rainbow (100y storage)": "Rainbow_100y",
    "CAR (storage component)": "CAR_storage",
}

plot_types = ["3D surface", "3D wireframe", "2D contour"]

metric_dd = Dropdown(options=list(metrics.keys()), value="Puro (CORCs)", description="Metric:", layout=Layout(width="320px"))
plot_dd = Dropdown(options=plot_types, value="3D surface", description="Plot:", layout=Layout(width="320px"))
show_points_cb = Checkbox(value=False, description="Show grid points")

def make_grids(df, value_col):
    H = np.sort(df["H_Corg"].unique())
    T = np.sort(df["soil_temp_C"].unique())
    H_grid, T_grid = np.meshgrid(H, T)
    Z = np.empty_like(H_grid, dtype=float)
    for i, Ts in enumerate(T):
        for j, hc in enumerate(H):
            Z[i, j] = df.loc[
                (df["soil_temp_C"] == Ts) & (df["H_Corg"] == hc),
                value_col
            ].values[0]
    return H_grid, T_grid, Z

def render(*args):
    value_col = metrics[metric_dd.value]
    H_grid, T_grid, Z = make_grids(df_sweep, value_col)

    with out:
        out.clear_output(wait=True)

        if plot_dd.value.startswith("3D"):
            fig = plt.figure()
            ax = fig.add_subplot(111, projection="3d")

            if plot_dd.value == "3D surface":
                ax.plot_surface(H_grid, T_grid, Z)
            else:
                ax.plot_wireframe(H_grid, T_grid, Z)

            if show_points_cb.value:
                ax.scatter(H_grid.ravel(), T_grid.ravel(), Z.ravel(), s=10)

            ax.set_xlabel("H/Corg")
            ax.set_ylabel("Soil temperature (°C)")
            ax.set_zlabel("Credits (tCO2e)")
            ax.set_title(metric_dd.value)
            plt.show()

        else:
            fig = plt.figure()
            ax = fig.add_subplot(111)
            cs = ax.contourf(H_grid, T_grid, Z)
            fig.colorbar(cs, ax=ax, label="Credits (tCO2e)")

            if show_points_cb.value:
                ax.scatter(H_grid.ravel(), T_grid.ravel(), s=10)

            ax.set_xlabel("H/Corg")
            ax.set_ylabel("Soil temperature (°C)")
            ax.set_title(metric_dd.value)
            plt.show()

metric_dd.observe(render, names="value")
plot_dd.observe(render, names="value")
show_points_cb.observe(render, names="value")

ui = VBox([HBox([metric_dd, plot_dd, show_points_cb]), out])
ui

render()
